In [1]:
# @copyright 2022, Shardul Rajhans.

# This is a research study for the course 'Master of Science in Data Science' performed by Shardul Rajhans 
# under the supervision of Dr Sandeep Raghuwanshi.

In [2]:
# Importing Necessary Libraries.

# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Importing matplotlib and seaborn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Standardize price values used across datasets
import re

In [3]:
df_list=[]

aerie = pd.read_csv("Dataset/ae_com.csv")
df_list.append(aerie)

calvin_klein = pd.read_csv("Dataset/calvinklein_com.csv")
df_list.append(calvin_klein)

amazon = pd.read_csv("Dataset/amazon_com.csv")
df_list.append(amazon)

btemptd = pd.read_csv("Dataset/btemptd_com.csv")
df_list.append(btemptd)

hanky_panky = pd.read_csv("Dataset/hankypanky_com.csv")
df_list.append(hanky_panky)

macys_com = pd.read_csv("Dataset/macys_com.csv")
df_list.append(macys_com)

nordstrom_com = pd.read_csv("Dataset/shop_nordstrom_com.csv")
df_list.append(nordstrom_com)

topshop = pd.read_csv("Dataset/us_topshop_com.csv")
df_list.append(topshop)

victoria_secret = pd.read_csv("Dataset/victoriassecret_com.csv")
df_list.append(victoria_secret)

In [4]:
fashion_data = pd.DataFrame()

for df in df_list:
    fashion_data = fashion_data.append(df)

In [5]:
# Printing shape
fashion_data.shape

(613143, 14)

In [6]:
# Displaying the information of all the columns
fashion_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 613143 entries, 0 to 453385
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   product_name      613143 non-null  object 
 1   mrp               613143 non-null  object 
 2   price             613143 non-null  object 
 3   pdp_url           613143 non-null  object 
 4   brand_name        613143 non-null  object 
 5   product_category  613143 non-null  object 
 6   retailer          613143 non-null  object 
 7   description       613143 non-null  object 
 8   rating            233556 non-null  float64
 9   review_count      222030 non-null  float64
 10  style_attributes  156675 non-null  object 
 11  total_sizes       613143 non-null  object 
 12  available_size    613143 non-null  object 
 13  color             612466 non-null  object 
dtypes: float64(2), object(12)
memory usage: 70.2+ MB


In [7]:
# Displaying the information of all the numeric columns
fashion_data.describe()

,rating,review_count
count,233556.000000,2.220300e+05
mean,4.254360,5.708913e+35
std,0.527576,9.170288e+36
min,0.000000,1.000000e+00
25%,4.000000,1.600000e+01
50%,4.300000,9.700000e+01
75%,4.500000,3.860000e+02
max,5.000000,1.560000e+38


# Cleaning the Dataset
In this section, we will analyze the missing values, remove the unnecessary columns, and perform all the necessary handling before Exploratory Data Analysis.

In [8]:
fashion_data.shape

(613143, 14)

In [9]:
# Drop the duplicate rows which might result in wrong results during EDA
fashion_data.drop_duplicates(inplace=True)

In [10]:
fashion_data.shape

(301026, 14)

In [11]:
# There are few columns that are not required throughout this study, hence removing the unnecessary columns.
columns_to_drop = ['pdp_url', 'retailer', 'rating', 'review_count', 'style_attributes']
fashion_data.drop(columns_to_drop, axis=1, inplace=True)
fashion_data.head()

,product_name,mrp,price,brand_name,product_category,description,total_sizes,available_size,color
0,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Rugged Green
1,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Natural Nude
2,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",True Black
3,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",White
4,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Royal Navy


In [12]:
# Conversion to USD for MRP and Price Coloumn
# Reference: https://www.kaggle.com/code/justinekays/eda-for-lingerie?scriptVersionId=8012060&cellId=14

colon_to_dollar_conversion_rate = 0.0017
ind_rp_to_dollar_conversion_rate = 0.000066

def extract_usd_value(value_str):
    if type(value_str) is str:
        value_str= value_str.strip().lower().replace('usd', '').replace('$','').replace("\-.*","").strip()
        value_str = re.sub(r"-.*", "", value_str)
        value_str = re.sub(r"–.*", "", value_str)
        value_str = re.sub(r"\s.*", "", value_str)
        value_str.strip()
        
        if "₡" in value_str:
            value_str = value_str.replace("₡", "").strip()
            value_str = pd.to_numeric(value_str) * colon_to_dollar_conversion_rate            
        elif "rp" in value_str:
            value_str = value_str.replace("rp", "").strip()
            value_str = pd.to_numeric(value_str) * ind_rp_to_dollar_conversion_rate                
            
        return value_str
    else:
        return value_str
    
fashion_data['mrp']=fashion_data['mrp'].apply(extract_usd_value).apply(pd.to_numeric)
fashion_data['price']=fashion_data['price'].apply(extract_usd_value).apply(pd.to_numeric)
fashion_data.head(10)

,product_name,mrp,price,brand_name,product_category,description,total_sizes,available_size,color
0,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Rugged Green
1,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Natural Nude
2,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",True Black
3,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",White
4,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Royal Navy
5,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Slab Gray
6,Aerie Lace Hi-Rise Boybrief,12.5,12.5,AERIE,Boybriefs,Laced with love and extra comfort. No cheek pe...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L""]",White
7,Aerie Lace Hi-Rise Boybrief,12.5,12.5,AERIE,Boybriefs,Laced with love and extra comfort. No cheek pe...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M""]",Natural Nude
8,Aerie Lace Hi-Rise Boybrief,12.5,12.5,AERIE,Boybriefs,Laced with love and extra comfort. No cheek pe...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S""]",True Black
9,Aerie Lace Hi-Rise Boybrief,12.5,12.5,AERIE,Boybriefs,Laced with love and extra comfort. No cheek pe...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M""]",Buff


In [13]:
fashion_data['color'] = fashion_data.color.str.lower()
fashion_data.head()

,product_name,mrp,price,brand_name,product_category,description,total_sizes,available_size,color
0,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",rugged green
1,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",natural nude
2,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",true black
3,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",white
4,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",royal navy


In [14]:
fashion_data['color'].unique().shape[0]

2282

In [15]:
# Categorizing the Products into Sub-Groups for generalization.
# Reference: https://www.kaggle.com/code/jkokatjuhha/data-driven-lingerie-shopping?scriptVersionId=6849300&cellId=16
def manipul_regex(str_array):
    string = '|'.join(str_array)
    return '(^|\s)(' + string + ')(\s\.|$)'

def categorize_product(data):
    panties = manipul_regex(['thong','g string','pant','v kini','boypant','pants','panty','thongs','panties',
                             'ladypant','knickers','thong','twist knicker','brief','boyshort',
                             'lace v front short','signature lace bike short','side tie bikini',
                             'signature lace string bikini','tanga','panty','hipster','vikini',
                             'cheekster', 'boypants','ladypants', 'boyshorts', 'hiphugger', 
                             'pink high leg logo bikini', 'pink shortie', 'pink logo bikini',
                             'lace sexy shortie', 'body base shorty', 'bikini bottom', 'lace mini bikini', 
                             'ruched mini bikini', 'high leg bikini'])  
    
    bodys = manipul_regex(['bodysuit', 'teddy', 'wink plaything', 'legging', 'dress'])
    bras = manipul_regex(['bra', 'bustier', 'strapless', 'balconette', 'bandeau', 'body by victoria unlined demi', 
                          'push-up', 'push up', 'push', 'pink seamless lightly lined racerback',
                          'body by wacoal seamless underwire','basic beauty wireless contour',
                          'pink lace lightly lined triangle', 'lace cross front unlined halter', 
                          'high neck keyhole halter', 'high-neck wrap'])
    
    activewear = manipul_regex(['sports bra', 'sport bra', 'sport bralette', 'sports bralette', 'strappybralette', 'full-zip'])
    suspenders = manipul_regex(['suspenders','belt'])
    bralettes = manipul_regex(['bralettes','bralette', 'bralet', 'silicone petal'])
    tops = manipul_regex(['tops','top','tee', 'tunic'])
    babydoll = manipul_regex(['babydoll','camisole and bikini set by bluebella','chemise'])
    shorts = manipul_regex(['short','shorts','chiffon tap pant'])
    slip = manipul_regex(['slip', 'waist slip', 'half-slip', 'slips', 'half-slips', 'petticoat'])
    robe = manipul_regex(['robe', 'kimonos','kimono'])
    camisole = manipul_regex(['camisole','cami','tank'])
    rompers = manipul_regex(['romper']) 
    onepiece = manipul_regex(['One-Piece', 'one piece', 'One-Piece Halter'])
    
    categories_list = [['slip',slip], ['shorts',shorts], ['robe', robe], ['tops',tops], ['suspenders',suspenders],
                 ['rompers', rompers], ['babydoll', babydoll], ['bodys',bodys], ['bralettes', bralettes], 
                 ['activewear', activewear], ['camisoles',camisole], ['bras',bras], ['panties',panties], ['onepiece', onepiece]]

    for items in categories_list:
        naming, reg = items
        data.loc[(data['product_name'].str.contains(reg, case=False)), 'product_category_wide'] = naming
    return data

In [16]:
def manipul_regex(str_array):
    string = '|'.join(str_array)
    return '(^|\s)(' + string + ')(\s|$)'

def categorize_colors(data):
    green = manipul_regex(["green", "emerald", "fir", "bayberry", "cocoon", "olive", "turquoise", "basil", 
                           "seafoam glow", "smokey pearl", "cactus"])
    blue = manipul_regex(["blue", "navy", "teal", "denim", "azure", "celeste", "turkish", "sea", "cloud", 
                          "neon", "ensign", "sky", "sapphire", "saphire", "blues", "ink", "aqua", "watercolor", 
                          "bluebell", "northstar", "bluebird", "inkblot", "nordic", "Evening Tide", "Marine"])
    white = manipul_regex(["white", "ivory", "cashew", "coconut", "marshmallow", "marble", "White/Victoria", 
                           "sugar", "suger", "moon", "Dragonfruit", "pillow", "white/rose", "White/Red"])
    red = manipul_regex(["red", "candy apple", "ginger glaze", "plum", "maroon", "ruby","cherry", "strawberry", 
                         "rose", "apple", "berry", "crimson"])
    yellow = manipul_regex(["yellow", "gold", "buff", "coral rock", 'daisy', "haze", "lime", "leopard", "bronzer", 
                            "ginger", "mango"])
    orange = manipul_regex(["orange", "peach", "sunset", "melon", "floral", "tropical", "teak", "peel", "fireworks", "firework"])
    pink = manipul_regex(["pink", "rosewater", "fuschia", "blush", "peach", "lotus", "fair orchid", "plum dust", "begonia", 
                          "hello lovely", "forever young", "lip", "lipsmacker", "dazzle", "geo", "celestial", 
                          "galaxy", "flamingo", "cream", "pinky", "mauvelous"])
    grey = manipul_regex(["gray", "grey", "pewter", "slate", "silver", "grey/blush", "heather", "Black/White", "marl", 
                          "shadow", "charcoal", "Marl/Victoria", "White/Black"])
    brown = manipul_regex(["brown", "taupe", "chai", "cappuccino", "sienna", "toast", "french roast"])
    maroon = manipul_regex(["maroon", "kir", "cinnamon"])
    purple = manipul_regex(["mauve", "lilac", "purple", "violet", "grape", "blackberry", "amethyst", "lavender", 
                            "lavendar", "red/Blue", "ignited", "eggplant", "fair orichid"])
    black = manipul_regex(["black", "midnight", "night", "Black/Victoria", "dark", "Sapphire/Black", "Black/Mesh", 
                           "Black/Zig Zag", "Black/", "black/rose", "Black/Rock", "Black/Red"])
    nude = manipul_regex(["nude", "bare", "champagne", "light", "neutral", "infinity", "dust", "flake", "Mint", 
                          "nightlife", "Snowflake", "cool", "clean", "Warm"])
    multicolor = manipul_regex(["multi", "blend", "blended", "radiating", "print", "stripe", "stripes", "chevron", 
                                "abstract", "colorblock", "spraypaint", "americana", "mesh", "spring", 
                                "french", "arrows", "arrrows", "moonray", "blocks", "curves", "coral", "rainbow", 
                                "metropolis", "border", "niagara", "aztec", "Striples", "bliss", "kaleidoscope", "party",
                               "paradise palm", "gradient", "2VN", "Seychelles", "multifolk", "C3K", "Multicolor"])
    
    colors_list = [['green',green], ['blue',blue], ['white', white], ['black',black], ['red',red],
                 ['yellow', yellow], ['orange', orange], ['pink',pink], ['nude', nude], 
                 ['grey', grey], ['brown',brown], ['maroon',maroon], ['purple',purple], ['multicolor', multicolor]]
    
    for items in colors_list:
        naming, reg = items
        data.loc[(data['color'].str.contains(reg, case=False)), 'color_group'] = naming
    return data

In [17]:
fashion_data.color.fillna('other', inplace=True)
fashion_data = categorize_colors(fashion_data)
fashion_data['color_group'].unique()

array(['green', 'nude', 'black', 'white', 'blue', 'grey', 'yellow', nan,
       'red', 'multicolor', 'maroon', 'pink', 'brown', 'purple', 'orange'],
      dtype=object)

In [18]:
fashion_data.product_category.fillna('other', inplace=True)
fashion_data = categorize_product(fashion_data)
fashion_data['product_category_wide'].unique()

array([nan, 'panties', 'bras', 'bralettes', 'babydoll', 'tops',
       'camisoles', 'shorts', 'rompers', 'bodys', 'suspenders', 'slip',
       'robe', 'activewear', 'onepiece'], dtype=object)

In [19]:
fashion_data['product_category_wide'].value_counts()

bras          215007
panties        42260
bralettes      26222
bodys           1529
babydoll        1291
slip            1204
camisoles       1052
onepiece         829
tops             805
rompers          200
shorts           174
activewear       170
robe             168
suspenders        40
Name: product_category_wide, dtype: int64

In [20]:
fashion_data['product_category_wide'].isnull().sum()

10075

In [21]:
# There are some products like bottle, basketball, etc. that are not relevant for this study, thus can be dropped.
# Also, the suspenders cateogory can be removed as it has only 2 entries.

fashion_data.product_category_wide.fillna('other', inplace=True)
fashion_data.color_group.fillna('other', inplace=True)
fashion_data.head()

,product_name,mrp,price,brand_name,product_category,description,total_sizes,available_size,color,color_group,product_category_wide
0,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",rugged green,green,other
1,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",natural nude,nude,other
2,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",true black,black,other
3,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",white,white,other
4,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",royal navy,blue,other


In [22]:
fashion_data.color_group.fillna('other', inplace=True)
fashion_data.head()

,product_name,mrp,price,brand_name,product_category,description,total_sizes,available_size,color,color_group,product_category_wide
0,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",rugged green,green,other
1,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",natural nude,nude,other
2,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",true black,black,other
3,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",white,white,other
4,Aerie Everyday Loves Lace Cheeky,12.5,12.5,AERIE,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",royal navy,blue,other


In [23]:
fashion_data['product_category_wide'].value_counts()

bras          215007
panties        42260
bralettes      26222
other          10075
bodys           1529
babydoll        1291
slip            1204
camisoles       1052
onepiece         829
tops             805
rompers          200
shorts           174
activewear       170
robe             168
suspenders        40
Name: product_category_wide, dtype: int64

In [24]:
def standardize_brand_names(brand_name):
    brand_name = brand_name.lower()
    brand_name = brand_name.replace("-", " ")
    
    if "hanky" in brand_name:
        brand_name = "hanky panky"
    elif "calvin" in brand_name:
        brand_name = "calvin klein"
    elif "wacoal" in brand_name or "tempt" in brand_name:
        brand_name = "b.tempt'd"
    elif "victoria" in brand_name:
        brand_name = "victoria's secret"
    elif "aeo" in brand_name:
        brand_name = "aerie"
    brand_name = brand_name.strip()
    return brand_name
fashion_data['brand_name'] = fashion_data['brand_name'].apply(standardize_brand_names)

In [25]:
fashion_data.brand_name.unique()

array(['aerie', 'calvin klein', "b.tempt'd", 'hanky panky',
       "victoria's secret", 's', 'fila', 'lucky brand', 'creative motion',
       'sexy hair', 'nintendo', 'compression comfort', 'vanity fair',
       'nordstrom lingerie', 'us topshop'], dtype=object)

# Feature Engineering

In [26]:
fashion_data.head()

,product_name,mrp,price,brand_name,product_category,description,total_sizes,available_size,color,color_group,product_category_wide
0,Aerie Everyday Loves Lace Cheeky,12.5,12.5,aerie,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",rugged green,green,other
1,Aerie Everyday Loves Lace Cheeky,12.5,12.5,aerie,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",natural nude,nude,other
2,Aerie Everyday Loves Lace Cheeky,12.5,12.5,aerie,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",true black,black,other
3,Aerie Everyday Loves Lace Cheeky,12.5,12.5,aerie,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",white,white,other
4,Aerie Everyday Loves Lace Cheeky,12.5,12.5,aerie,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",royal navy,blue,other


In [27]:
fashion_data.shape

(301026, 11)

In [28]:
victoria_secret = fashion_data[fashion_data['brand_name']=="victoria's secret"]
victoria_secret.shape

(239104, 11)

In [29]:
# Data except victoria's secret
brands_data = fashion_data[fashion_data['brand_name'] != "victoria's secret"]
brands_data.head()

,product_name,mrp,price,brand_name,product_category,description,total_sizes,available_size,color,color_group,product_category_wide
0,Aerie Everyday Loves Lace Cheeky,12.5,12.5,aerie,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",rugged green,green,other
1,Aerie Everyday Loves Lace Cheeky,12.5,12.5,aerie,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",natural nude,nude,other
2,Aerie Everyday Loves Lace Cheeky,12.5,12.5,aerie,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",true black,black,other
3,Aerie Everyday Loves Lace Cheeky,12.5,12.5,aerie,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",white,white,other
4,Aerie Everyday Loves Lace Cheeky,12.5,12.5,aerie,Cheekies,Introducing Everyday Loves™: Made with love. E...,"[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",royal navy,blue,other


In [30]:
brands_data = brands_data.groupby(['product_name', 'brand_name', 'product_category', 
                                   'description', 'product_category_wide']).agg({'mean'}).reset_index()
brands_data.head()

,product_name,brand_name,product_category,description,product_category_wide,mrp,price
,,,,,,mean,mean
0,"""I DO"" Signature Lace Cheeky Hipster with Gift...",hanky panky,Collections,"With ""I DO"" emblazoned on this Hanky Panky lac...",panties,40.0,40.0
1,"""I DO"" Signature Lace Cheeky Hipster with Gift...",hanky panky,Panties,"With ""I DO"" emblazoned on this Hanky Panky lac...",panties,40.0,40.0
2,'After Midnight' Lace Open Gusset G-String,hanky panky,Bridal Lingerie,Stretch signature lace fashions an alluring do...,other,27.0,27.0
3,'After Midnight' Lace Open Gusset G-String,hanky panky,Sexy Lingerie,Stretch signature lace fashions an alluring do...,other,27.0,27.0
4,'After Midnight' Lace Open Gusset G-String,hanky panky,Women's Panties,Stretch signature lace fashions an alluring do...,other,27.0,27.0


In [31]:
brands_data.columns = ['product_name', 'brand_name', 'product_category', 'description', 'product_category_wide', 'mrp', 'price']
brands_data['%discount'] = round((1-(brands_data.price/brands_data.mrp))*100, 2)
brands_data.head()

,product_name,brand_name,product_category,description,product_category_wide,mrp,price,%discount
0,"""I DO"" Signature Lace Cheeky Hipster with Gift...",hanky panky,Collections,"With ""I DO"" emblazoned on this Hanky Panky lac...",panties,40.0,40.0,0.0
1,"""I DO"" Signature Lace Cheeky Hipster with Gift...",hanky panky,Panties,"With ""I DO"" emblazoned on this Hanky Panky lac...",panties,40.0,40.0,0.0
2,'After Midnight' Lace Open Gusset G-String,hanky panky,Bridal Lingerie,Stretch signature lace fashions an alluring do...,other,27.0,27.0,0.0
3,'After Midnight' Lace Open Gusset G-String,hanky panky,Sexy Lingerie,Stretch signature lace fashions an alluring do...,other,27.0,27.0,0.0
4,'After Midnight' Lace Open Gusset G-String,hanky panky,Women's Panties,Stretch signature lace fashions an alluring do...,other,27.0,27.0,0.0


In [32]:
brands_data.to_csv('Dataset/Intermediate/brands_data_comp.csv')